### Title: Bollinger Bands Backtest (Pandas)
Author: Tan Zhi Lun  
Contact: zhilun296@gmail.com

### The strategy
This is a simple mean reversion strategy, where:
1. A short order will be placed if the price closes above the Upper Bollinger Band
2. A long order will be placed if the price closes below the Lower Bollinger Band

The timeframe for the strategy is 1 minute, and the dataset used for backtesting is 1 hr in length in total. However, this can easily be extended to longer timeframes as well.

In [2]:
import pandas as pd
import numpy as np

#read file into pd dataframe
filename = "EURUSD_out_2017_1hr.csv"
df = pd.read_csv(filename)

#Reduce columns that are not needed
del df['Number']
del df['Open']
del df['High']
del df['Low']

# Strategy refers to BBands, create Mean, Low and High columns for the MA and bands respectively
def Strategy(DF, n):
    df = DF.copy()
    df['Mean'] = df['Close'].rolling(n).mean()
    df['BBands_Diff'] = df['Close'].rolling(n).std() * 2
    df['BBands_High'] = df['Mean'] + df['BBands_Diff']
    df['BBands_Low'] = df['Mean'] - df['BBands_Diff']
    del df['BBands_Diff']
    return df
df = Strategy(df,20)

# Function to find last occurrence of 1, -1 and 2 in 'Position' column. 
# Emptylist to keep track of index.
# Emptylist2 and Emptylist 3 to keep track of order type.
# These columns will be added in later on in the code.
def get_last_one(DF,n):
    emptylist = []
    emptylist2=[]
    emptylist3 = [0]
    for i in range(n):
        if DF.loc[i,'Position'] == 1:
            emptylist.append(i)
            emptylist2.append(1)
        elif DF.loc[i,'Position'] == -1:
            emptylist.append(i)
            emptylist2.append(-1)
        elif DF.loc[i,'Position'] == 2:
            emptylist3.append(i)
    return (emptylist[-1],emptylist2[-1],emptylist3[-1])

print(df[21:30])

               datetime    Close      Mean  BBands_High  BBands_Low
21  2017-04-28 21:21:00  1.08983  1.089743     1.090046    1.089441
22  2017-04-28 21:22:00  1.08982  1.089739     1.090036    1.089443
23  2017-04-28 21:23:00  1.08983  1.089736     1.090028    1.089445
24  2017-04-28 21:24:00  1.08984  1.089734     1.090022    1.089447
25  2017-04-28 21:25:00  1.08982  1.089730     1.090010    1.089451
26  2017-04-28 21:26:00  1.08979  1.089724     1.089991    1.089457
27  2017-04-28 21:27:00  1.08979  1.089727     1.089996    1.089458
28  2017-04-28 21:28:00  1.08972  1.089739     1.089981    1.089497
29  2017-04-28 21:29:00  1.08973  1.089756     1.089931    1.089582


In [21]:
# To determine position, 1 means buy, then will continuously hold, -1 is sell, 0 is no position, 2 means close trade
df['Position1'] = 0
df['Position'] = 0

# Buy and Sell respectively
for i in df.index:
    if df.loc[i,'Close'] <= df.loc[i,'BBands_Low']:
        df.loc[i,'Position1'] = 1
    elif df.loc[i,'Close'] >= df.loc[i,'BBands_High']:
        df.loc[i,'Position1'] = -1
    elif i>=1 and df.loc[i-1,'Position1'] == 1 and \
        (df.loc[i,'Mean']-df.loc[i,'Close'])>=0.0002:
        df.loc[i,'Position1'] = 1
    elif i>=1 and df.loc[i-1,'Position1'] == -1 and \
        (df.loc[i,'Close']-df.loc[i,'Mean'])>=0.0002:
        df.loc[i,'Position1'] = -1        

# As an example demonstration only, we would not know the index where a position is entered in practice
# Where 'Position1' is 1, a long order is entered and held until 'Position1' is no longer 1
print (df[['datetime','Close','BBands_High','BBands_Low','Position1']][30:44])

               datetime    Close  BBands_High  BBands_Low  Position1
30  2017-04-28 21:30:00  1.08968     1.089907    1.089621          0
31  2017-04-28 21:31:00  1.08956     1.089905    1.089624          1
32  2017-04-28 21:32:00  1.08944     1.089952    1.089554          1
33  2017-04-28 21:33:00  1.08944     1.089980    1.089495          1
34  2017-04-28 21:34:00  1.08943     1.089993    1.089443          1
35  2017-04-28 21:35:00  1.08933     1.090018    1.089372          1
36  2017-04-28 21:36:00  1.08932     1.090034    1.089311          1
37  2017-04-28 21:37:00  1.08942     1.090035    1.089280          1
38  2017-04-28 21:38:00  1.08931     1.090041    1.089230          1
39  2017-04-28 21:39:00  1.08931     1.090036    1.089188          1
40  2017-04-28 21:40:00  1.08926     1.090022    1.089145          1
41  2017-04-28 21:41:00  1.08921     1.090005    1.089100          1
42  2017-04-28 21:42:00  1.08924     1.089978    1.089069          1
43  2017-04-28 21:43:00  1.08943  

In [22]:
# Adjust so that in 'Position' column there will be a visible 'transition' from holding to selling
for i in df.index:
    if i>=1 and df.loc[i,'Position1'] == 1 and df.loc[i-1,'Position1'] ==0:
        df.loc[i,'Position'] =1
    if i>=1 and df.loc[i,'Position1'] == -1 and df.loc[i-1,'Position1'] ==0:
        df.loc[i,'Position'] =-1
    elif i>=1 and df.loc[i,'Position1'] == 0 and df.loc[i-1,'Position1'] !=0:
        df.loc[i,'Position'] =2
df['Position'] = df['Position'].astype(np.int)
del df['Position1']

# As an example demonstration only, we would not know the index where a position is entered in practice
# Where 'Position' is 1, a long order is entered, and 'Position' is 2 means position is exited
print (df[30:44])

               datetime    Close      Mean  BBands_High  BBands_Low  Position
30  2017-04-28 21:30:00  1.08968  1.089764     1.089907    1.089621         0
31  2017-04-28 21:31:00  1.08956  1.089764     1.089905    1.089624         1
32  2017-04-28 21:32:00  1.08944  1.089753     1.089952    1.089554         0
33  2017-04-28 21:33:00  1.08944  1.089737     1.089980    1.089495         0
34  2017-04-28 21:34:00  1.08943  1.089718     1.089993    1.089443         0
35  2017-04-28 21:35:00  1.08933  1.089695     1.090018    1.089372         0
36  2017-04-28 21:36:00  1.08932  1.089672     1.090034    1.089311         0
37  2017-04-28 21:37:00  1.08942  1.089657     1.090035    1.089280         0
38  2017-04-28 21:38:00  1.08931  1.089635     1.090041    1.089230         0
39  2017-04-28 21:39:00  1.08931  1.089612     1.090036    1.089188         0
40  2017-04-28 21:40:00  1.08926  1.089584     1.090022    1.089145         0
41  2017-04-28 21:41:00  1.08921  1.089553     1.090005    1.089

In [23]:
#To determine pip difference
df['Pip Diff'] = 0

# get_last_one function is used here, this is to tell which reference points should be used to calculate pip difference.
for i in df.index:
    if df.loc[i,'Position'] == 2:
        magic_index = get_last_one(df,i)[0]
        magic_buysell = get_last_one(df,i)[1]
        if magic_buysell == 1:
            df.loc[i,'Pip Diff'] = df.loc[i,'Close'] -df.loc[magic_index,'Close']
        elif magic_buysell == -1:
            df.loc[i,'Pip Diff'] = df.loc[magic_index,'Close'] -df.loc[i,'Close']

# As an example demonstration only, we would not know the index where a position is entered in practice
print (df[['datetime','Close','BBands_High','BBands_Low','Pip Diff']][30:44])

               datetime    Close  BBands_High  BBands_Low  Pip Diff
30  2017-04-28 21:30:00  1.08968     1.089907    1.089621   0.00000
31  2017-04-28 21:31:00  1.08956     1.089905    1.089624   0.00000
32  2017-04-28 21:32:00  1.08944     1.089952    1.089554   0.00000
33  2017-04-28 21:33:00  1.08944     1.089980    1.089495   0.00000
34  2017-04-28 21:34:00  1.08943     1.089993    1.089443   0.00000
35  2017-04-28 21:35:00  1.08933     1.090018    1.089372   0.00000
36  2017-04-28 21:36:00  1.08932     1.090034    1.089311   0.00000
37  2017-04-28 21:37:00  1.08942     1.090035    1.089280   0.00000
38  2017-04-28 21:38:00  1.08931     1.090041    1.089230   0.00000
39  2017-04-28 21:39:00  1.08931     1.090036    1.089188   0.00000
40  2017-04-28 21:40:00  1.08926     1.090022    1.089145   0.00000
41  2017-04-28 21:41:00  1.08921     1.090005    1.089100   0.00000
42  2017-04-28 21:42:00  1.08924     1.089978    1.089069   0.00000
43  2017-04-28 21:43:00  1.08943     1.089936   

In [27]:
# Simulation of a trader bot, parameters as outlined here
df['Position Size'] = 0
df['Profit'] = 0
df['Commission'] = 0
df['Balance'] = 100000

# Calculate cumulative balance
for i in df.index:
    if i>=1:
        df.loc[i,'Balance'] = df.loc[i-1,'Balance']
        
    # Calculate position size (based on entry, thus get_last_one function is required)
    if df.loc[i,'Position'] == 2:
        magic_index2 = get_last_one(df,i)[0]
        df.loc[i, 'Position Size'] = round(df.loc[magic_index2, 'Balance']*0.02/df.loc[magic_index2,'Close']/1000,2)
        
        # Calculate profit
        df['Profit'] = df['Pip Diff'] * df['Position Size'] * 100000
        
        # Calculate commission, assumed that there is no bid ask spread but rather commission is a flat rate at 3.76%
        # This is based on a broker called Pepperstone, which offers flat commission instead of bid ask spread
        df.loc[i,'Commission'] = df.loc[i,'Position Size']*3.76*-2

    df.loc[i,'Balance'] = df.loc[i,'Balance'] + df.loc[i,'Profit'] + df.loc[i,'Commission']
    
# As always, only for illustration purposes as we would not know when position is entered
# There are 2 positions entered during the hour
print("First Order")
print(df[['datetime','Position Size', 'Profit', 'Commission', 'Balance']][30:45])
print(" ")
print("Second Order")
print(df[['datetime','Position Size', 'Profit', 'Commission', 'Balance']][55:60])

First Order
               datetime  Position Size  Profit  Commission      Balance
30  2017-04-28 21:30:00           0.00    0.00      0.0000  100000.0000
31  2017-04-28 21:31:00           0.00    0.00      0.0000  100000.0000
32  2017-04-28 21:32:00           0.00    0.00      0.0000  100000.0000
33  2017-04-28 21:33:00           0.00    0.00      0.0000  100000.0000
34  2017-04-28 21:34:00           0.00    0.00      0.0000  100000.0000
35  2017-04-28 21:35:00           0.00    0.00      0.0000  100000.0000
36  2017-04-28 21:36:00           0.00    0.00      0.0000  100000.0000
37  2017-04-28 21:37:00           0.00    0.00      0.0000  100000.0000
38  2017-04-28 21:38:00           0.00    0.00      0.0000  100000.0000
39  2017-04-28 21:39:00           0.00    0.00      0.0000  100000.0000
40  2017-04-28 21:40:00           0.00    0.00      0.0000  100000.0000
41  2017-04-28 21:41:00           0.00    0.00      0.0000  100000.0000
42  2017-04-28 21:42:00           0.00    0.00      

In [28]:
# Check for open positions at the end, as we would like to calculate equity, not the raw balance in the account
if get_last_one(df,df.index[-1])[2] < get_last_one(df,df.index[-1])[0]:
    print("Warning: Final Position Not CLosed!")
else:
    print("All positions closed.")

# Calculate total number of trades entered
asum = 0
for i in df.index:
    if df.loc[i,'Position'] == 2:
        asum += 1
        
# Print out final metrics of the backtest
print("Number of trades taken is:",asum)
print("Final Balance is", round(df.loc[df.index[-1],'Balance'],2))

All positions closed.
Number of trades taken is: 2
Final Balance is 99959.45
